# Demo of running a Flyte + Feast, feature engineering and training pipeline
In this demo we will learn how to interact with Feast through Flyte. The goal will be to train a simple [Gaussian Naive Bayes model using sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) on the [Horse-Colic dataset from UCI](https://archive.ics.uci.edu/ml/datasets/Horse+Colic).
The model aims to classify if the lesion of the horse is surgical or not. This is a modified version of the original dataset.

**NOTE**
We will not really dive into the dataset or the model, as the aim of this tutorial is to show how you can use Feast as the feature store and use Flyte to engineer the features that can be identical across your online predictions as well as offline training

## Step 1: Check out the code for the pipeline
We have used [flytekit](https://docs.flyte.org/projects/flytekit/en/latest/) flyte's python SDK to express the pipeline in pure python. The actual workflow code is auto-documented and rendered using sphinx [here](https://flyte--424.org.readthedocs.build/projects/cookbook/en/424/auto/case_studies/feature_engineering/feast_integration/index.html) *to be merged soon*

## Step 2: Launch an execution
We can use the [FlyteConsole](https://github.com/flyteorg/flyteconsole) to launch, monitor and introspect Flyte executions, but in this case we will use [flytekit.remote](https://docs.flyte.org/projects/flytekit/en/latest/design/control_plane.html) to interact with the Flyte backend.

### Setup flytekit remote from config
To work with flytesandbox, we have created a simple local config that points to FlyteSandbox server and execution environment. We will initialize flytekit remote with this server. We will also pin it to one project and domain.

**Note** this also sets up access to S3 or other equivalent datastores needed by FEAST

In [ ]:
from flytekit.remote import FlyteRemote
remote = FlyteRemote.from_config("flytesnacks", "development")

### Retrieve the latest registered version of the pipeline
FlyteRemote provides convienient methods to retrieve a version of the pipeline from the remote server.

**Note** It is possible to get a specific version of workflow and trigger a launch for that, but, we will just get the latest

In [ ]:
# from feast_integration.feast_workflow import feast_workflow
lp = remote.fetch_launch_plan(name="feast_integration.feast_workflow.feast_workflow")
lp.id.version

### Launch an execution
`remote.execute` makes it simple to start an execution for the launchplan. We will not provide any inputs and just use the default inputs

In [ ]:
exe = remote.execute(lp, inputs={})
print(f"http://localhost:30081/console/projects/{exe.id.project}/domains/{exe.id.domain}/executions/{exe.id.name}")

## Step 3: Now wait for the execution to complete
It is possible to launch a sync execution and wait for it to complete, but since all the processes are completely detached (you can even close your laptop) and come back to it later, we will show how to sync the execution back.

In [ ]:
from flytekit.models.core.execution import WorkflowExecutionPhase
exe = remote.sync(exe)
print(f"Execution {exe.id.name} is in Phase - {WorkflowExecutionPhase.enum_to_string(exe.closure.phase)}")

In [ ]:
exe.sync()

## Step 4: Lets sync data from this execution

**Side Note**
It is possible to fetch an existing execution or simply retrieve a started execution. Also if you launch an execution with the same name, flyte will respect and not restart a new execution!

To fetch an execution
```python
exe = remote.fetch_workflow_execution(name='f9f180a56e67b4c9781e')
exe = remote.sync(exe)
```

In [ ]:
from feast_dataobjects import FeatureStore
fs = exe.raw_outputs.get('o0', FeatureStore)
model = exe.outputs['o1']

#### Lets inspect the feature store configuration

In [ ]:
fs.config

#### Also, the model is now available locally as a JobLibSerialized file and can be downloaded and loaded

In [ ]:
model

## Step 5: Cool, Let's predict
So we have the model and a feature store!, how can you run predictions. Flytekit will automatically manage the IO for you and you can simply re-use the prediction function from the workflow.

### Lets load some features from the online feature store
We are re-using the feature definition from the flyte workflow
```python
inference_point = fs.get_online_features(FEAST_FEATURES, [{"Hospital Number": "533738"}])
```

In [ ]:
from feast_workflow import predict, FEAST_FEATURES
inference_point = fs.get_online_features(FEAST_FEATURES, [{"Hospital Number": "533738"}])
inference_point

### Now run a prediction
Notice how we are passing the serialized model and some loaded features

In [ ]:
predict(model_ser=model, features=inference_point)

## Done! 
We can ofcourse observe the intermediates from the workflow, which we saw in the UI, we can also download any intermediate data.

## Future
We want to further improve this experience, to allow for the same prediction method to run in your inference server and in a workflow. It is almost there now, but you need to remove the `model de-serialization` as this happens for the current predict method.